In [ ]:
import full_model
import numpy as np

In [ ]:
full_model.eng = full_model.start_matlab()
show_plot = False

In [ ]:
from optimize import *

In [ ]:
gp = optimize(gp_minimize, get_params(2), 13, 0, verb_model=True)

In [ ]:
from skopt import load

gp_loaded = load("optims/scenario" + str(3) + "/gp_" + str(0) + "_" + str(50) + ".opt")
gp = reload(gp_loaded, get_params(3), 3, 0, verb_model=True)

In [10]:
tests = {'fun', 'space', 'x', 'func_vals', 'x_iters'}
def test(val1, val2):
    assert np.alltrue([np.alltrue(val1[key] == val2[key]) for key in tests]), "TESTS FAILED"
    print("ALL TESTS PASSED!")

In [ ]:
used_func = forest_minimize
seed = 1234
res1 = optimize(used_func, 12, 1, 1, seed)
res1_copy = deepcopy(res1)
res12 = reload(res1, 4, seed)
res23 = reload(res12, 4, seed)
res13 = reload(res1, 8, seed)
res3 = optimize(used_func, 20, 1, 1, seed)

from skopt import dump, load
dump(res1, "temp.txt")
res1_loaded = load("temp.txt")
res12_loaded = reload(res1_loaded, 4, seed)

In [97]:
test(res13, res3)
test(res23, res3)
test(res1, res1_loaded)
test(res12, res12_loaded)
test(res1_copy, res1)

ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
